# Realizando uma Analise para Scalping Trading Strategy

## Intro

Navegando na internet vi um artigo que fizia que este metodo no qual vamos analisar abaixo era efetivo em 74% das vezes, o que chamou atencao, como pode um metodo tao simples ser tao efetivo, é a chance de qualquer pessoa ficar rica...Pensei.

Mas não é bem assim... Vejamos.

`Scalping Trading Strategy` consiste em uma estrategia simples de compra de venda de acoes baseada em condicoes simples que sao listadas abaixo:

- Nos `entramos no jogo` quando: O pregão abre com um aumento de 1% em relação ao preço de fechamento do `dia anterior`.

- Nos `saimos do jogo` quando: Existir um aumento de mais de 1% sobre o valor de compra. Se a aç~ao falhar em subir 1% sobre o valor de compra nos saimos ao final do dia.

Enfim, a meta é não criar estrategias mirabolantes...

Vamos tentar analisar se essa estrategia é vantajosa, se sua simplicidade nos daria algum retorno financeiro e vamos mais alem tentando mostrar, quanto seria esse retorno financeiro.


## Importando Bibliotecas

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from termcolor import colored as cl
import numpy as np
import yfinance as yf


## Definindo algumas variaveis

In [14]:
#ESTE É O PERCENTUAL EM QUE CONSIDERAMOS PARA ENTRAR NO JOGO
VAR_P_CLOSE_P_OPEN = 3 # %

#ESTE É O PERCENTUAL EM QUE SE VENDERIA AS ACOES...NO CASO DELA SUBIR X% EM RELACAO AO 
#PRECO DE ABERTURA
VAR_P_CHANGE = 1 # %

titulos = ['ITUB4.SA','PETR3.SA','AAPL34.SA','GGBR4.SA']
periodo = '2y' 


## Extraindo Dados

Extrair os dados sera muito importante para para nossa analize e vamos utilizar o `yfinanece` para obter os dados.

O Código abaixo realiza a coleta destes dados e os mostra.

In [15]:
titulo = titulos[2]
stock = pd.DataFrame(yf.Ticker(titulo).history(period = periodo))
stock.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-04-18 00:00:00-03:00,38.339142,38.492498,37.770238,38.091793,370814,0.0,0.0
2022-04-19 00:00:00-03:00,38.091792,38.779423,37.730661,38.388611,329904,0.0,0.0
2022-04-20 00:00:00-03:00,38.863523,39.026775,37.987908,38.170948,280070,0.0,0.0
2022-04-22 00:00:00-03:00,38.586490,39.150446,38.388610,38.497444,389402,0.0,0.0
2022-04-25 00:00:00-03:00,38.418292,39.313696,38.230304,39.293907,350992,0.0,0.0


## Calculando a % de Mudança e Aplicando Filtro

De acordo com nossa estrategia de entrar no mercado somente se o valor de abertura for 1% maior que o dia anterior, precisamos fazer esse calculo... 

Para isso usaremos o codigo a seguir...

In [17]:
stock['p_close_open'] = np.nan # criando uma coluna NAN

for i in range(1, len(stock)):
    diff = (stock['Close'].iloc[i-1] - stock['Open'].iloc[i])
    # avg = (stock['Close'].iloc[i-1] + stock['Open'].iloc[i])/2
    avg = stock['Close'].iloc[i-1] 
    pct_change = (diff / avg) * 100
    #armazena o vlaor encontrado
    stock.loc[stock.index[i] ,'p_close_open'] = pct_change


# filtra os resultados caso o % encontrado dia anterior valor de abertura for igual ao desejado
#no caso do exemplo 1%
stock = stock[stock['p_close_open'] > VAR_P_CLOSE_P_OPEN]

n_vezes_p_abertura_maior = stock['p_close_open'].count()
print(n_vezes_p_abertura_maior)

stock.head()



4


,Open,High,Low,Close,Volume,Dividends,Stock Splits,p_close_open
Date,,,,,,,,
2022-11-03 00:00:00-03:00,36.266852,36.599256,35.274599,35.324211,581856,0.0,0.0,5.224946
2023-08-04 00:00:00-03:00,45.007719,45.326710,44.160395,44.260082,525596,0.0,0.0,3.628595
2024-02-02 00:00:00-03:00,44.460266,46.448491,44.320392,46.218693,287047,0.0,0.0,3.700498
2024-02-14 00:00:00-03:00,45.450001,45.900002,45.369999,45.900002,217062,0.0,0.0,3.008963


## Testando Estrategia

Com os dados em mãos, vamos verificar a eficacia de nossa estrategia. Utilizaremos os dados obtidos para verificar em quais porcentagem de casos seriamos eficazes.

Primeiro vamos contar quantos dias obtivemos um % de ganho igual a 2%

In [18]:
stock['aum_percentual_dia'] = np.nan # criando uma coluna NAN

for i in range(1, len(stock)):

    diff = (stock['High'].iloc[i] - stock['Open'].iloc[i])
    
    pct_change = (diff / stock['Open'].iloc[i]) * 100
    # print(f"Max: {stock['High'].iloc[i]} , Open: {stock['Open'].iloc[i]} , % {pct_change}")
    #armazena o vlaor encontrado
    stock.loc[stock.index[i] ,'aum_percentual_dia'] = pct_change

n_vezes_p_abertura_plus = stock[stock['aum_percentual_dia'] > VAR_P_CLOSE_P_OPEN + VAR_P_CHANGE ]['aum_percentual_dia'].count()

Varificar quantos dias houve um ganho qualquer no fechamento.

In [19]:
n_vezes_ganho = stock[stock['Close'] > stock['Open']]['Close'].\
    count()
print(n_vezes_ganho)

2


## Conclusao...

Para esse papel em especifico a estrategia adotada resulta em um ganho improvavel...

In [20]:
print(f'Informação para o titulo {titulo}')
print(f'Numero de DIAS que valor de abertura maior que {VAR_P_CLOSE_P_OPEN}% : {n_vezes_p_abertura_maior}')
print(f'Numero de DIAS com VALOR MAX maior que {VAR_P_CLOSE_P_OPEN + VAR_P_CHANGE }% : {n_vezes_p_abertura_plus}')
print(f'Uma porcentagem de {n_vezes_p_abertura_plus / n_vezes_p_abertura_maior * 100 :.2f}%')
print(f'Numero de DIAS que houve um ganho quaquer no valor de fechamento : {n_vezes_ganho}')
print(f'Uma porcentagem de {n_vezes_ganho / n_vezes_p_abertura_maior * 100 :.2f}%')


Informação para o titulo AAPL34.SA
Numero de DIAS que valor de abertura maior que 3% : 4
Numero de DIAS com VALOR MAX maior que 4% : 1
Uma porcentagem de 25.00%
Numero de DIAS que houve um ganho quaquer no valor de fechamento : 2
Uma porcentagem de 50.00%


No poximo arquivo vamos sondar quais as acoes, dentro de todas as acoes brasileiras, qual podemos aplicar tal metodologia.

O arquivo `Listar_acoes.py` faz uma analize de todas as açcoes do mercado Brasileiro e retona um data frame com as acoes que possuem uma real probabilidade de ganho.

Para um valor de abertura de 1% e um rendimento de 0.5% temos as seguintes acoes...

```python
0   ARML3.SA           63            46       73.015873                44   69.841270
1   BMLC11.SA           63            48       76.190476                52   82.539683
2   BPML11.SA           94            72       76.595745                84   89.361702
3   BRBI11.SA           48            34       70.833333                30   62.500000
4    CLSA3.SA           71            56       78.873239                45   63.380282
5    EVEN3.SA           39            31       79.487179                26   66.666667
6    HBSA3.SA           38            28       73.684211                23   60.526316
7   HOFC11.SA           30            23       76.666667                26   86.666667
8    LOGN3.SA           60            45       75.000000                41   68.333333
9   NEWU11.SA           72            52       72.222222                57   79.166667
10  PAGS34.SA           74            54       72.972973                46   62.162162
11   PLPL3.SA           58            49       84.482759                35   60.344828
12   ROMI3.SA           30            23       76.666667                22   73.333333
13   TRIS3.SA           48            36       75.000000                29   60.416667
14   VLID3.SA           62            44       70.967742                39   62.903226
15   VULC3.SA           42            30       71.428571                31   73.809524
16  WALM34.SA           47            38       80.851064                41   87.234043
```